In [1]:
# ====================== 环境准备 ======================
# 1) 设定环境变量（必须在导入 matplotlib 之前执行）
import os

from draw.basic_functio.get_rectangular_size_interval import calc_envelope_for_group

os.environ['QT_API'] = 'pyqt5'        # 指定使用 PyQt5 作为 Qt 绑定
os.environ['MPLBACKEND'] = 'QtAgg'    # 指定 Matplotlib 后端为 QtAgg（更推荐，替代 TkAgg）

# 2) 启动 Qt 事件循环（控制台模式下，让 Qt 窗口能实时响应）
%gui qt5

# 3) 检查 matplotlib backend
import matplotlib as mpl
mpl.rcParams.update({
    # 这里按系统常见字体给一串候选，存在则自动生效
    "font.sans-serif": ["Microsoft YaHei", "SimHei", "SimSun",
                        "Noto Sans CJK SC", "Source Han Sans SC",
                        "Arial Unicode MS", "DejaVu Sans"],
    "font.family": "sans-serif",
    "axes.unicode_minus": False,   # 负号用正常字符，避免被当作缺字形
})

print("backend (before pyplot):", mpl.get_backend())
# 如果不是 QtAgg，强制改为 QtAgg（注意：必须在导入 pyplot 前设置）
mpl.rcParams['backend'] = 'QtAgg'

# 4) 现在再导入 pyplot
import matplotlib.pyplot as plt
print("backend (after pyplot):", mpl.get_backend())



backend (before pyplot): module://matplotlib_inline.backend_inline
backend (after pyplot): QtAgg


In [2]:

# ====================== 导入依赖 ======================
import sys
# 避免反复执行时 Qt 类重复导入导致崩溃：如果已加载，先删除再导入
if 'draw.pyqt_draw.pyqt_main2' in sys.modules:
    del sys.modules['draw.pyqt_draw.pyqt_main2']

from PyQt5 import QtWidgets
import pyqtgraph as pg
from draw.pyqt_draw.pyqt_main2 import SatelliteViewer
import draw.read_snap_xml as read_snap_xml
import draw.read_snap_xml as read_snap_xml
# 配置 pyqtgraph：开启抗锯齿，关闭 OpenGL（更稳定）
pg.setConfigOptions(antialias=True)

In [3]:
# ====================== 基础参数 ======================
# 星座参数：每轨道卫星数 N，轨道平面数 P
N = 36
P = 18

# ====================== 读取数据 ======================
xml_file = r"E:\Data\station_visible_satellites_648_1d_real.xml"
start_ts = 0
# end_ts   = 86399
end_ts   = 1024
# 解析 XML 得到 group_data，结构：{time_step: {'groups': {...}}}
group_data = read_snap_xml.parse_xml_group_data(xml_file, start_ts, end_ts)

#下面是图变换的。
#

In [4]:
rev_group_data,offset = read_snap_xml.modify_group_data(group_data, N=36, groupid=4)

下面主要是为了测试检测我们的图

In [5]:

# ====================== 绘图初始化 ======================
# 1) QApplication 实例（全局唯一）
app = QtWidgets.QApplication.instance() or QtWidgets.QApplication([])

# 2) 确保 viewer 有全局引用，避免 GC 回收导致崩溃
if not hasattr(sys.modules[__name__], "_viewer_list"):
    _viewer_list = []

In [ ]:


# 3) 创建并配置 viewer
viewer = SatelliteViewer(group_data)
viewer.setWindowTitle("group_data")
viewer.resize(1200, 700)
# viewer.edges_by_step =
# viewer.pending_links_by_step =
viewer.show()



In [ ]:
# 3) 创建并配置 viewer
viewer = SatelliteViewer(rev_group_data)
viewer.setWindowTitle("rev_group_data")
viewer.resize(1200, 700)
# viewer.edges_by_step =
# viewer.pending_links_by_step =
viewer.show()


In [ ]:
# 4) 保存全局引用,只要放入到这个容器里，就能持久存在
_viewer_list.append(viewer)

In [6]:

# get the rectangular size of the group
# 同样是计算block尺寸
import draw.basic_functio.get_rectangular_size_interval as get_rectangular_size_interval
t1,t2=get_rectangular_size_interval.calc_envelope_for_group(rev_group_data,[start_ts,end_ts],0,P,N)
t3,t4=get_rectangular_size_interval.calc_envelope_for_group(rev_group_data,[start_ts,end_ts],4,P,N)

下面主要是为了确认，我们的包络矩形是否正确

In [7]:

# ====================== 静态包络矩形 ======================
#可选
# rects = {
#     4: [(9, 15, 32, 35)],
#     0: [(0, 5, 9, 13), (17, 17, 30, 32)],
# }
rects = {
    0: (t1, t2),
    4: (t3, t4),
}



In [ ]:
colors = {4: "deeppink", 0: "orange"}  # 可选

viewer.show_envelopes_static(
    rects_by_group=rects,
    expand=0.35,
    colors=colors,
    persist=True
)

## 微调时间区间
早些时候的时间区间是通过包络大大小去确定的，但是我们发现，包络矩形的位置也在变化，有时候单纯以大的包络矩形来覆盖位置变化不够精确，因此，在这一步，我们往往会根据包络矩形位置以及大小进一步的细分调整时间

In [ ]:
df = get_rectangular_size_interval.get_envelope_traces(rev_group_data, groupid=0, P=18, N=36, t0=start_ts, t1=end_ts)

# 2. 可直接保存df到csv
# df.to_csv('envelope_trace_group4.csv', index=False)

# 3. 画随时间变化的包络范围
get_rectangular_size_interval.plot_envelope_traces(df)

In [ ]:
import  importlib
importlib.reload(get_rectangular_size_interval)

## 同构图规划
确定好包络矩形后，我们就可以开始规划同构图拓扑了

In [ ]:

##仅仅为test，不一定运行
# 生成无需 step 的边
edges = {}
for i in range(P - 1):
    for j in range(N):
        nownode = i * N + j
        next_node1 = (i + 1) * N + j
        edges.setdefault(nownode, set()).add(next_node1)
        upnodes = i * N + (j + 1) % N
        edges.setdefault(nownode, set()).add(upnodes)
        downnodes = i * N + (j - 1 + N) % N
        edges.setdefault(nownode, set()).add(downnodes)

In [8]:

# 这里一般就是规划的地方了，这里进行手动规划，
nodes = {}

import draw.basic_functio.motif as motif
# 下面就是motif图案填充，重复图案填充。里面有多种选项。
# 1
# motif.write_distinct_motif(0, 17, 9, 35, P, N, nodes, option=0)
# motif.write_distinct_motif(0, 17, 0, 8, P, N, nodes, option=1)
motif.write_distinct_motif(0, 17, 9, 35, P, N, nodes, option=0)
motif.write_distinct_motif(0, 17, 0, 8, P, N, nodes, option=1)

rev_inter_edge = motif.transform_nodes_2_adjacent(nodes,P,N)

C:\Users\yfh\anaconda3\envs\graph_ga\Lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


下面是同构图验证，只需要查看一个拓扑图即可，

In [11]:


# 3) 下面是同构图设计，我们一般从同构图上设计出 motif，然后，再迁移到我们其他图的显示上去
import draw.pyqt_draw.pyqt_onetopology as pyqt_onetopology
viewer = pyqt_onetopology.Onetopology(rects)
viewer.setWindowTitle("Grouped Satellite Visibility - High Performance (PyQtGraph)")
viewer.resize(1200, 700)
motif_construct_edge = rev_inter_edge
viewer.edges_by_step = motif_construct_edge
viewer.show()
_viewer_list.append(viewer)

In [9]:

# 将同构图的拓扑设计映射到该时间段的同构拓扑设计上去，
all_rev_inter_edge = {}
for i in range(start_ts,end_ts):
    all_rev_inter_edge[i] = rev_inter_edge
# here the adj can be used for the qt5 to draw

### 处理冲突链路
上述获得的是原始图的边，但是，其存在冲突问题，例如 t=100s是a与b t=101s就是a与c了，考虑到链路建链需要时间，因此，我们这里需要进行一些冲突修正
假设建链时间是setuptime  =60s
1. 若a与b是区域内链路，a与c是是区域外链路，由于区域内链路是必须保留的，因此，我们认为，a与c的链路是从t=101才开始建链，并且t=160末尾才完成建链，并在t=161时投入使用
2. 若a与b是区域外链路，a与c是区域内链路，由于区域内链路是必须保留的，因此，我们认为，a与c的链路在t=101时刻就可投入使用，因此，a与b的链路已经在前60s断链，认为a与c的链路在t=101-60也就是y=41s时开始建链



In [10]:

# 注意上述我们是在同构拓扑序列上进行的，因此，我们还要将同构拓扑序列进行转化，同时，我们还要考虑到建链时间约束
import draw.basic_functio.revdata2rawdata as revdata2rawdata
# attention ,here  it just composed of the inter-link, intra_link hasn't benn conclued
raw_inter_edge = revdata2rawdata.revedge2rawedge(all_rev_inter_edge,offset)

In [60]:
viewer = SatelliteViewer(group_data)
viewer.setWindowTitle("groupdata with rawedge")
viewer.resize(1200, 700)
viewer.edges_by_step =raw_inter_edge

viewer.show()


In [51]:
# 4) 保存全局引用,只要放入到这个容器里，就能持久存在
_viewer_list.append(viewer)

In [49]:
viewer = SatelliteViewer(rev_group_data)
viewer.setWindowTitle("rev_group_data with rev_group_data")
viewer.resize(1200, 700)
viewer.edges_by_step = all_rev_inter_edge

viewer.show()
_viewer_list.append(viewer)

In [76]:

import draw.basic_functio.motif as motif
edges_by_step = motif.transform_nodes_2_rawedge(nodes, P, N, start_ts, end_ts)
from draw.pyqt_draw.pyqt_main2 import SatelliteViewer


# 3) 创建并配置 viewer
viewer = SatelliteViewer(group_data)
viewer.setWindowTitle("group_data")
viewer.resize(1200, 700)
viewer.edges_by_step =edges_by_step
# viewer.pending_links_by_step =
viewer.show()
_viewer_list.append(viewer)

我们要处理好建链时间冲突，因此，下面就是处理冲突的代码

In [13]:
import importlib
import draw.basic_functio.conflict_link as conflict_link
importlib.reload(conflict_link)
# 建链时间约束
time_2_build = 60

# raw_edges_by_step, pending_links_by_step = conflict_link.get_no_conflict_link(raw_inter_edge,group_data,start_ts,end_ts,time_2_build,N,P)

raw_edges_by_step,pending_edges = conflict_link.get_no_conflict_link(raw_inter_edge,offset,rects,start_ts,end_ts,time_2_build,N,P)

In [12]:
viewer = SatelliteViewer(group_data)
viewer.setWindowTitle("groupdata with rawedge")
viewer.resize(1200, 700)
viewer.edges_by_step =raw_edges_by_step

viewer.show()

In [ ]:
import draw.basic_functio.inter_edge2nodes as inter_edge2nodes
all_inter_edge = inter_edge2nodes.trans_edge2node(raw_edges_by_step,P,N)

In [14]:

# 下面是将实际拓扑进行显示，用于查看效果


viewer = SatelliteViewer(group_data)
viewer.setWindowTitle("groupdata with rawedge")
viewer.resize(1200, 700)
viewer.edges_by_step =raw_edges_by_step
viewer.pending_links_by_step =pending_edges
viewer.show()


In [ ]:
##在检查实际拓扑后，确认无问题后，就将其存入到xml文件里去，注意，我们只要保存异轨链路信息即可，其余不必保存
import draw.basic_functio.write2xml as write2xml
import genaric2.tegnode as tegnode
# 推荐：用 raw string 防止反斜杠转义，并改成有意义的文件名
# 这里，我们要把原始的边转为node进行存储
all_nodes = inter_edge2nodes.trans_edge2node(raw_edges_by_step,P,N)

write2xml.nodes_to_xml(
    all_nodes,
    rf"E:\研究生\研究进展\工作记录\实验记录\interplane_links_{start_ts}_{end_ts}.xml"
)
